In [1]:
import os

from db import VecDBClient
from utils import get_text_embedding
from dotenv import load_dotenv
import openai

load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")

client = VecDBClient(
    url="http://localhost:6333",
    collection_name="inflex_document",
    embedding_dim=1536
)

query = "员工要遵守什么规则"
query_embedding = get_text_embedding(query, embedding_model_name="gpt-3.5-turbo")

points = client.retrieve_similar_vectors(query_embedding, top_k=1)

for item in points[0]:
    print(item)

metadata
page_content


In [3]:
points[0]['page_content']

'员工守则\n\n一.基本要求\n\n熟悉岗位职责；明确工作目标；\n\n遵循业务流程；遵守作业标准；\n\n注意团队和谐；维护企业形象；\n\n保守商业秘密；不断开拓进取。\n\n二.员工行为准则\n\n1.热爱本职工作，遵守职业道德，尽职尽责，服从领导，听从指挥，严守纪律；\n\n2.遵纪守法，严禁有碍社会公德及伤害他人名誉的行为；\n\n3.为人诚实，不隐瞒，伪造自身经历，不掩饰工作上的过失和错误；\n\n4.不滥用职权和越权专断，不得利用职权索取谋求其他不正当利益；\n\n5.维护公司信誉，非经许可不得擅用本公司名义对外发表个人意见；\n\n6.本公司员工对外接洽事项，应态度谦和，不得骄傲自满；\n\n7.员工对于职务及公事，均应循级而上，不得越级呈报，但紧急或特殊情况不在此限。\n\n三.员工录用\n\n※面试\n\n1.新员工入职之前需要填写入职申请表；\n\n2.如实告知本人的基本信息提供身份证原件、学历证、健康证原件及联系方式；\n\n3.本人过往的经历、家庭状况及特殊情况下的联络人的电话；\n\n4.面试先要经过人力资源部初试，然后由招聘需求的部门主管根据业务的需要进一步复试；'

# summerize

In [27]:
dict = {
    "question": "",
    "options": {
        "A": "",
        "B": "",
        "C": "",
        "D": "",
    },
    "answer": ""
}
"""
{{
    "question": ```your generated question```
    "options": {{
    "A": ```Option A content```,
    "B": ```Option B content```,
    "C": ```Option C content```,
    "D": ```Option D content```,
    }},
    "answer": ```The correct answer of your generated question```
}}
"""

# summarize knowledge points

In [24]:
from dotenv import load_dotenv
import openai
import os

load_dotenv()
openai.api_key = os.environ.get("OPENAI_API_KEY")

with open("../document_txt/人力资源部/ZC-S-H-002RBA管理手册（A1）.txt", "r") as f:
    text = f.read()

prompt_1 = """
You have been assigned the responsibility of overseeing staff training and there is a company article available for your reference. \
Your task is to analysis the article and extract {NUM_KNOWLEDGE_POINT} key knowledge mentioned in the article that you believe are crucial for employees, \
and describe each knowledge in a comprehensive summary in Chinese with about 100 words. \
Please mark each knowledge clearly with bullet point.

Output example: ```
-- <your generated knowledge name> | <your generated knowledge summary>
```
Article: ```{ARTICLE}```
""".format(NUM_KNOWLEDGE_POINT=2, ARTICLE=text[600:1200])

response_gpt = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt_1,
    temperature=0.7,
    max_tokens=2048,
    n=1,
    top_p=1,
)
summarized_knowledge_points = response_gpt.choices[0].text

from pprint import pprint
pprint(summarized_knowledge_points)
# summarized_knowledge_points

('\n'
 '-- 线路板制造经验 | '
 '这篇文章介绍了该公司在板及刚挠性印制线路板批量和样板制造方面的经验和能力。他们具有较强及快速的新品/新工艺开发能力，可以确保快速提供优质的高科技产品，广泛应用于通信、工业控制、计算机应用、航空航天、军工、医疗、测试仪器等各个领域。\n'
 '\n'
 '-- 禁止使用童工政策 | '
 '该公司制定了政策和补救措施来有效地控制不使用童工。这个政策适用于公司内的各个部门和工作场所，并且遵守国务院的相关规定和RBA标准。在招工时，人力资源部门必须对应聘者的相关证件进行核查，确认其身份和年龄符合要求后才能录用。如果应聘者还未领取身份证，必须持有户口所在地公安派出所核准的身份证明。这些措施旨在保护未成年人的权益，遵守相关法律法规。')


# generate questions

In [21]:
prompt_2 = """
You are the person in charge of employee orientation, and you have a list of knowledge points. \
Your task is to generate a multiple-choice question for each knowledge point to see \
how well the employee has mastered each of the knowledge point. \
Each multiple-choice question has 1 correct option and 3 interference options\
(must ensure the interference option is erroneous). 
Please generate {NUM_QUESTION} questions and output each question in key-value format, make sure key name is English and value content is Chinese. 

Output Example:
```
question: <your generated question>
options: A: <Option A content> | B: <Option B content> | C: <Option C content> | D: <Option D content>\n
answer: <The correct option of your generated question>
<DIVIDED>
```

Knowledge points: ```{KNOWLEDGE_POINTS}```
""".format(NUM_QUESTION=2, KNOWLEDGE_POINTS=summarized_knowledge_points)

response_gpt = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt_2,
    temperature=0.7,
    max_tokens=2048,
    n=1,
    top_p=1,
)
raw_questions = response_gpt.choices[0].text

from pprint import pprint
pprint(raw_questions)

('\n'
 'question: 按照公司政策，哪些领域适用于该公司快速提供优质的高科技产品？\n'
 'options: A: 医疗 | B: 农业 | C: 通信 | D: 资源开发\n'
 '\n'
 'answer: C: 通信\n'
 '<DIVIDED>\n'
 '\n'
 'question: 公司人力资源部在招工时会进行哪些核查？\n'
 'options: A: 核对相关证件 | B: 进行面试 | C: 查看求职者简历 | D: 向求职者提问\n'
 '\n'
 'answer: A: 核对相关证件')


In [5]:
def parse_raw_questions_to_json(raw_questions: str) -> list:
    
    processed_question = []
    for raw_question in raw_questions.split('<DIVIDED>'):
        raw_question = raw_question.strip().split('\n')
        json_question = {}
        for section in raw_question:
            if section.startswith("question"):
                question_content = section.split("question:")[1].strip()
                json_question["question"] = question_content
            
            elif section.startswith("options"):
                json_options = {}
                for option in section.split("options:")[1].split("|"):
                    option_tag = option.split(":")[0].strip()
                    option_content = option.split(":")[1].strip()
                    json_options[option_tag] = option_content
                json_question["options"] = json_options
            
            elif section.startswith("answer"):
                answer_content = section.split("answer:")[1].strip()
                json_question["answer"] = answer_content
            
        processed_question.append(json_question)
    return processed_question

parse_raw_questions_to_json(raw_questions)

[{'question': '公司名称为广东则成科技有限公司的经营理念是？',
  'options': {'A': '以市场为导向，以质量为中心',
   'B': '立足行业领先地位',
   'C': '追求高精密度生产',
   'D': '持续改善生产制造和服务'},
  'answer': 'A: 以市场为导向，以质量为中心'},
 {'question': 'RBA管理手册是公司为控制社会责任及安全卫生损失而建立的指导性文件，该手册是根据哪些标准和法规编写的？',
  'options': {'A': 'RBA实施及内容审核',
   'B': 'ISO9001：2008质量管理体系',
   'C': 'ISO14OO0：2004环境管理体系',
   'D': '所有以上内容'},
  'answer': 'D: 所有以上内容'}]

In [30]:
with open('ZC-S-H-002知识点.txt', 'r', encoding='utf-8') as txt_f:
    knowledge_text = txt_f.read()

knowledge_list = knowledge_text.replace("--", "\n"). replace("\n\n", "\n").replace("\n\n", "\n").strip().split("\n")
knowledge_list


['RBA管理手册 | 从文章中，我们可以了解到RBA管理手册是为我司的社会责任及安全卫生损失而建立的管理方案。它的实施内容参照了ISO9001：2008质量管理体系、ISO14OO0：2004环境管理体系、责任商业联盟行为准则（RBA）及国家、地方相关法律法规。通过定期的符合性评审，它能够改善公司与员工及相关利益团体的关系，提升企业形象及声誉，增强员工的向心力、凝聚力和公司竞争力。',
 ' 公司简介 | 文章中提到，广东则成科技有限公司是一家座落于广东省珠海市高新技术产业开发区的港资企业，专业生产高精密双面、多层挠性板及刚挠结合板。公司的经营理念是以“市场为导向，质量为中心”，致力于高精密度批量和快速印制线路板的生产制造和研发服务，力争实现行业领先。通过构建与客户关系紧密、利益取向一致的价值链体系，公司团队具有近二十年的行业经验。',
 ' 童工政策 | 本公司制定了严格的童工政策，遵守国务院《禁止使用童工规定》和RBA标准。招聘时，人力资源部必须核查应聘人员的相关证件，如第二代身份证和健康证，并确认相片真实无伪。暂未领取身份证的，应持有公安派出所核准的十六周岁以上的证明。',
 ' 厂房布局 | 公司的厂房分布图包括办公室、制造车间、仓库和实验室。制造车间包括开料车间、数控车间、电镀车间、快压车间、压合车间、线路车间、阻焊车间、冲切车间、CCD冲孔车间、终检车间、包装车间、AOI车间和电测车间。实验室包括物理室和化验室。这些车间和实验室的布局合理，能够有效地支持公司的生产和研发工作。',
 ' 未成年工特殊保护制度 | 根据《劳动法》，制定本制度，保护未成年工的合法权益和健康。',
 ' 《禁止使用童工规定》 | 根据规定，未成年工必须年满16周岁，不能从事危险、不安全和不卫生的工作，每年进行健康检查。',
 ' 作业分级 | 该公司严格按照国家标准对作业进行分级，包括接尘作业、有毒作业、冷水作业、高温作业和体力劳动强度作业等。同时，对于易燃易爆、化学性烧伤和热烧伤等危险性大的作业，也要特别注意。',
 ' 未成年工健康检查 | 公司要求对未成年工定期进行健康检查，包括安排工作岗位前、工作满1年以及年满18周岁且距前一次体检超过半年等条件。根据检查结果，对不能胜任原劳动岗位的未成年工，要根据医务部门的证明，减轻劳动量或安排其他劳动。',
 ' 员工权利保障 |

In [41]:
def parse_raw_questions_to_json(raw_questions: str) -> list[dict]:
    """
    Parse ChatGPT generated string content, and convert each generated question --to--> json format
    :param raw_questions: string of ChatGPT response
    :return: list of question in json format
    """
    processed_question = []
    for raw_question in raw_questions.split('<DIVIDED>'):
        raw_question = raw_question.split('\n')
        json_question = {}
        for section in raw_question:
            section = section.replace(" ", "")
            if section.startswith("question"):
                question_content = section.split("question:")[1]
                json_question["question"] = question_content

            elif section.startswith("options"):
                json_options = {}
                for option in section.split("options:")[1].split("|"):
                    option_tag = option.split(":")[0]
                    option_content = option.split(":")[1]
                    json_options[option_tag] = option_content
                json_question["options"] = json_options

            elif section.startswith("answer"):
                answer_content = section.split("answer:")[1]
                json_question["answer"] = answer_content
        
        if len(json_question) == 3:
            processed_question.append(json_question)
        
    return processed_question

In [42]:
knowledge_text = """
    question: RBA管理手册是为了哪些目的而建立的？
    options: A: 保证质量管理体系 | B: 保证环境管理体系 | C: 保证安全卫生损失 | D:保证社会责任
    answer: D: 保证社会责任
    <DIVIDED>
"""

parse_raw_questions_to_json(knowledge_text)

[{'question': 'RBA管理手册是为了哪些目的而建立的？',
  'options': {'A': '保证质量管理体系',
   'B': '保证环境管理体系',
   'C': '保证安全卫生损失',
   'D': '保证社会责任'},
  'answer': 'D:保证社会责任'}]